## San Franscisco to Portland Flight Path Optimization

This project used the data from the [OpenFlight](https://openflights.org/data.html) airport and routes dataset and aimed to find all paths between SFO and PDX. Sorted by distance.

In [1]:
import pandas as pd
import math
import networkx as nx
from networkx import drawing
from networkx import graph
import heapq

Importing packages needed for the project.
    - Math is imported to handle calculations
    - Networkx is imported to handle the graph
    - Heapq is imported to handle the priority queue 
    - Pandas is imported to read the text data

In [2]:
ap_col = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Lat',
          'Log', 'Alt', 'Timezone', 'DST', 'TZ Database', 'Type', 'Source']
airports = pd.read_csv('./data/airports.dat.txt', header=None, names=ap_col)


rt_col = ['Airline', 'Airline ID', 'Source Airport', 'Src Airport ID',
          'Dest Airport', 'Dest Airport ID', 'Codeshare', 'Stops', 'Equipment']
routes = pd.read_csv('./data/routes.dat.txt', header=None, names=rt_col)

First we created a variable (ap_col and rt_col) to store an array of column names. Then we used the pandas packaged to read and store the text files into variables (airports and routes).

### Data Exploration

In [3]:
airports.info()
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7698 entries, 0 to 7697
Data columns (total 14 columns):
Airport ID     7698 non-null int64
Name           7698 non-null object
City           7649 non-null object
Country        7698 non-null object
IATA           7698 non-null object
ICAO           7698 non-null object
Lat            7698 non-null float64
Log            7698 non-null float64
Alt            7698 non-null int64
Timezone       7698 non-null object
DST            7698 non-null object
TZ Database    7698 non-null object
Type           7698 non-null object
Source         7698 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 842.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 9 columns):
Airline            67663 non-null object
Airline ID         67663 non-null object
Source Airport     67663 non-null object
Src Airport ID     67663 non-null object
Dest Airport       67663 non-null object
Dest Airpor

The first step is to explore the data.  We used the info function to see what colums we had and what data types we were working with.

In [4]:
airports.isnull().sum()

Airport ID      0
Name            0
City           49
Country         0
IATA            0
ICAO            0
Lat             0
Log             0
Alt             0
Timezone        0
DST             0
TZ Database     0
Type            0
Source          0
dtype: int64

In [5]:
routes.isnull().sum()

Airline                0
Airline ID             0
Source Airport         0
Src Airport ID         0
Dest Airport           0
Dest Airport ID        0
Codeshare          53066
Stops                  0
Equipment             18
dtype: int64

We checked to see which columns in the airports dataframe had a value of null and summed that value.

In [6]:
route_array = routes['Source Airport']
lengths_4 = len([len(x) for x in route_array if len(x) == 4])
print(lengths_4)

0


In [7]:
dest_array = routes['Dest Airport']
ap_lengths_4 = len([len(x) for x in dest_array if len(x) == 4])
print(ap_lengths_4)

0


We then checked to see how many of the airport ids were 4 digits, because the documentation said that the id could be either have 3 or 4 digits. We discovered that the IATA codes were only 3 digits, so we didn't need to take the other codes into consideration.

In [8]:
dest_airport = routes['Dest Airport'] == 'SFO'
src_airport = routes['Source Airport'] == 'PDX'
condition = dest_airport & src_airport
newDF = routes[condition]
print(newDF)

      Airline Airline ID Source Airport Src Airport ID Dest Airport  \
6385       AA         24            PDX           3720          SFO   
11843      AS        439            PDX           3720          SFO   
57438      UA       5209            PDX           3720          SFO   
62023      VX       5331            PDX           3720          SFO   

      Dest Airport ID Codeshare  Stops    Equipment  
6385             3469         Y      0      DH4 737  
11843            3469       NaN      0      734 73G  
57438            3469       NaN      0  319 739 320  
62023            3469       NaN      0          320  


We ran a search on all nonstop flights between SFO and PDX to determine which airlines we wanted to filter for to limit computation resources.

### Solution

In [9]:
def toRadians(degrees):
    return (degrees * math.pi)/180.0

This function converts degrees to radians.

In [10]:
def distance(lat1, lon1, lat2, lon2):
    # R radius of Earth
    R = 6371e3  # metres
    #phi is latitude
    lat1_rad = toRadians(lat1)
    lat2_rad = toRadians(lat2)
    # difference lat
    delta_lat = toRadians((lat2-lat1))
    # difference lon
    delta_lon = toRadians((lon2-lon1))

    a = math.sin(delta_lat/2) * math.sin(delta_lat/2) + math.cos(lat1_rad) * \
        math.cos(lat2_rad) * math.sin(delta_lon/2) * math.sin(delta_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    d = R * c
    return math.trunc(d/1000)

This function calculates the distance between two coordinates based on the curvature of the Earth. Returns distance in kilometers with decimels truncated. 

In [11]:
def flightplanner(source, destination):
    G = nx.Graph()
    newDF = routes[(routes.Airline == 'AS') | (routes.Airline == 'AA')]
    for index, airport in airports.iterrows():
        G.add_node(airport['IATA'])
    for i, route in newDF.iterrows():
        route_dest = route["Dest Airport"]
        route_src = route["Source Airport"]
        if (route_dest in airports['IATA'].array) and (route_src in airports['IATA'].array):
            src_lat = airports[airports.IATA == route_src].iloc[0]['Lat']
            src_log = airports[airports.IATA == route_src].iloc[0]['Log']

            dest_lat = airports[airports.IATA == route_dest].iloc[0]['Lat']
            dest_log = airports[airports.IATA == route_dest].iloc[0]['Log']

            route_dist = distance(src_lat, src_log, dest_lat, dest_log)

            G.add_edge(route_src, route_dest, weight=route_dist)

    return G

This function takes the source and destination airports by their three digit codes and creates a graph object with all airports as nodes. The routes are pre-filtered to include only Alaska Airlines and American Airlines to limit computation time. All pre-filtered routes are then added to the graph as edges, weighted by the distance between the two airports. It returns the populated graph object (G).

In [12]:
def path_distance(G, array_of_stops):
    distance = 0
    for index in range(0, len(array_of_stops)-1):
        distance = distance + G.edges[array_of_stops[index], array_of_stops[index + 1]]['weight']

    return distance

This function takes in the populated graph object and an array of stops and returns the total distance along the path.

In [13]:
def weighted(G, paths):
    h = []
    for path in paths:
        distance = path_distance(G, path)
        t = (distance, path)
        heapq.heappush(h, t)

    return [heapq.heappop(h) for index in range(len(h))]

This function takes in the populated graph object and paths and returns a priority queue of tuples that includes the distance and stop array. It is then sorted from shortest to longest distance. Distance is what determines the tuple's priority. It returns the sorted array by utilizing heappop.

In [14]:
G = flightplanner('SFO', 'PDX')

coolflight = nx.all_simple_paths(G, source='SFO', target='PDX', cutoff=3)

paths = list(coolflight)

The flightplanner function output takes in the source and destination arguments and stores the returns the graph object which is stored in variable G. 
The all_simple_paths function takes in G, source and target nodes, with a cutoff of 3 stops and returns the array of paths, which is stored in variable coolflight. Coolflight is then converted to a list and stored in the paths variable.

In [16]:
distance_route_sorted = weighted(G, paths)

df = pd.DataFrame(distance_route_sorted, columns=['Distance', 'Stops'])

print(df.head(20))

    Distance                 Stops
0        886            [SFO, PDX]
1       1301       [SFO, SEA, PDX]
2       1599  [SFO, SEA, BLI, PDX]
3       1638  [SFO, SEA, EUG, PDX]
4       1646  [SFO, SEA, RDM, PDX]
5       1647  [SFO, SEA, PSC, PDX]
6       1700  [SFO, SEA, YVR, PDX]
7       1886       [SFO, LAX, PDX]
8       1887  [SFO, LAX, FAT, PDX]
9       1887  [SFO, LAX, RNO, PDX]
10      1898  [SFO, SEA, GEG, PDX]
11      1900  [SFO, LAX, RDM, PDX]
12      1913  [SFO, LAX, SMF, PDX]
13      1916  [SFO, LAX, MFR, PDX]
14      1917  [SFO, LAX, EUG, PDX]
15      1954  [SFO, LAX, SJC, PDX]
16      1973  [SFO, LAX, STS, PDX]
17      2017  [SFO, SEA, MFR, PDX]
18      2082       [SFO, PSP, PDX]
19      2149  [SFO, LAX, LAS, PDX]


The weighted function is then called to organize the paths calculated, which is then stored in the variable, distance_route_sorted.
The data is then converted into a dataframe for readability, capped at the top 10 shortest paths. Enjoy!